In [1]:
from datasets import load_dataset

ds = load_dataset("M-AI-C/quran_tafseer")

README.md:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

c:\Users\Lenovo\ML Notebooks\RAG\v_env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\datasets--M-AI-C--quran_tafseer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


(…)-00000-of-00001-41442851e73a7bb3.parquet:   0%|          | 0.00/9.63M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6235 [00:00<?, ? examples/s]

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['en-ahmedali', 'en-ahmedraza', 'en-arberry', 'en-asad', 'en-daryabadi', 'en-hilali', 'en-itani', 'en-maududi', 'en-mubarakpuri', 'en-pickthall', 'en-qarai', 'en-qaribullah', 'en-sahih', 'en-sarwar', 'en-shakir', 'en-transliterati', 'en-wahiduddi', 'en-yusufali', 'ayah', 'sorah', 'sentence'],
        num_rows: 6235
    })
})

In [9]:
from datasets import load_dataset

ds_hadis = load_dataset("M-AI-C/all_hadith")
ds_hadis

README.md:   0%|          | 0.00/581 [00:00<?, ?B/s]

c:\Users\Lenovo\ML Notebooks\RAG\v_env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\datasets--M-AI-C--all_hadith. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


(…)-00000-of-00001-6b65a39e54b5260f.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/34441 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'chapter_no', 'hadith_no', 'chapter', 'chain_indx', 'text_ar', 'text_en'],
        num_rows: 34441
    })
})

In [10]:
import faiss
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from typing import List


In [11]:
from datasets import load_dataset

quran_df = load_dataset("M-AI-C/quran_tafseer")
hadith_df = load_dataset("M-AI-C/all_hadith")


In [12]:
hadith_df

DatasetDict({
    train: Dataset({
        features: ['source', 'chapter_no', 'hadith_no', 'chapter', 'chain_indx', 'text_ar', 'text_en'],
        num_rows: 34441
    })
})

In [13]:
quran_df

DatasetDict({
    train: Dataset({
        features: ['en-ahmedali', 'en-ahmedraza', 'en-arberry', 'en-asad', 'en-daryabadi', 'en-hilali', 'en-itani', 'en-maududi', 'en-mubarakpuri', 'en-pickthall', 'en-qarai', 'en-qaribullah', 'en-sahih', 'en-sarwar', 'en-shakir', 'en-transliterati', 'en-wahiduddi', 'en-yusufali', 'ayah', 'sorah', 'sentence'],
        num_rows: 6235
    })
})

In [19]:
from langchain.schema import Document

# Function to preprocess Quran dataset with metadata
def preprocess_quran_data_with_metadata(dataset):
    df = dataset.to_pandas()
    
    translation_cols = [
        'en-ahmedali', 'en-ahmedraza', 'en-arberry', 'en-asad', 'en-daryabadi', 
        'en-hilali', 'en-itani', 'en-maududi', 'en-mubarakpuri', 'en-pickthall', 
        'en-qarai', 'en-qaribullah', 'en-sahih', 'en-sarwar', 'en-shakir', 
        'en-transliterati', 'en-wahiduddi', 'en-yusufali'
    ]
    
    df['knowledge'] = df[translation_cols].apply(lambda row: ' '.join(row.dropna()), axis=1)
    
    # Filter out rows where 'knowledge' is empty or None
    df = df[df['knowledge'].notna() & (df['knowledge'] != '')]
    
    return [
        Document(
            page_content=row['knowledge'],
            metadata={"ayah": row['ayah'], "sorah": row['sorah']}
        )
        for _, row in df.iterrows()
    ]

# Function to preprocess Hadith dataset with metadata
def preprocess_hadith_data_with_metadata(dataset):
    df = dataset.to_pandas()
    
    # Filter out rows where 'text_en' is empty or None
    df = df[df['text_en'].notna() & (df['text_en'] != '')]
    
    return [
        Document(
            page_content=row['text_en'],
            metadata={
                "source": row['source'],
                "chapter_no": row['chapter_no'],
                "hadith_no": row['hadith_no'],
                "chapter": row['chapter']
            }
        )
        for _, row in df.iterrows()
    ]

# Convert Hugging Face datasets to processed documents
quran_documents = preprocess_quran_data_with_metadata(quran_df['train'])
hadith_documents = preprocess_hadith_data_with_metadata(hadith_df['train'])
documents = quran_documents + hadith_documents


In [20]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Create the FAISS index with metadata-capable documents
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_store = FAISS.from_documents(documents, embedding_model)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12688\1991956341.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Lenovo\ML Notebooks\RAG\v_env\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline

# Load tokenizer and model for the generative pipeline
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

# Create a generation pipeline
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300, temperature=0)
llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="Answer the question based on the following context:\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"
)


In [ ]:
from langchain.chains import RetrievalQA

# Set up the RetrievalQA chain
retrieval_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    prompt_template=prompt_template
)


In [ ]:
def answer_question_with_metadata(query: str, retriever, qa_chain, top_k=5) -> str:
    docs = retriever.get_relevant_documents(query)[:top_k]
    
    # Combine document contents and reference metadata for context
    context = "\n\n".join(
        [f"{doc.page_content} (Reference: {doc.metadata})" for doc in docs]
    )
    
    # Run the LLM chain with the combined context
    response = qa_chain.run(context=context, question=query)
    return response




In [ ]:
# Example question
user_query = "What guidance does the Quran give about patience?"
response = answer_question_with_metadata(user_query, vector_store.as_retriever(), retrieval_qa_chain)
print(response)